In [1]:
import pandas as pd

In [2]:
norm_rates_file = 'bpm_norm_rates.csv'
data = pd.read_csv(norm_rates_file)

print(data.head())

In [3]:
print(data.tail())

In [3]:
# Extracting BPM min and BPM max columns
bpm_min = data['BPM min']
bpm_max = data['BPM max']

# Display the first few entries of each
print(f'BPM min: {bpm_min.head()}')
print(f'BPM max: {bpm_max.head()}')

## Usage BPM Min and Max Values

Depending on what you need these values for, there are several operations you might consider:

1. Statistical Analysis: You can compute statistical values such as the average, median, standard deviation, etc., for these BPM rates.
2. Visualizing Data: You might want to plot these BPM rates to see trends or outliers.
python
3. Further Data Processing: If you are preparing to use these BPM rates for machine learning or more detailed analysis, you might want to filter, normalize, or scale these values based on your requirements.
4. Correlation Analysis: Determine if there's a correlation between the minimum and maximum BPM rates or with other variables in your dataset.

In [4]:
# 1. Statistical Analysis: You can compute statistical values such as the average, median, standard deviation, etc., for these BPM rates.

print("Statistics for BPM Min:")
print(bpm_min.describe())

print("\nStatistics for BPM Max:")
print(bpm_max.describe())

In [5]:
# 2. Visualizing Data: You might want to plot these BPM rates to see trends or outliers.

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
bpm_min.plot(title='BPM Min Rates')
plt.subplot(1, 2, 2)
bpm_max.plot(title='BPM Max Rates')
plt.tight_layout()
plt.show()


In [6]:
# 3. Example: Normalize BPM rates
bpm_min_normalized = (bpm_min - bpm_min.min()) / (bpm_min.max() - bpm_min.min())
bpm_max_normalized = (bpm_max - bpm_max.min()) / (bpm_max.max() - bpm_max.min())

print("Normalized BPM Min Rates:")
print(bpm_min_normalized.head())

In [7]:
# Correlation Analysis

correlation = data[['BPM min', 'BPM max']].corr()
print("Correlation between BPM Min and Max:")
print(correlation)


In [2]:
import pandas as pd

# Load your data
data = pd.read_csv('C:/Users/benz/SynologyDrive/PycharmProjects/ToxitracAI/datasets/bpm_norm_rates.csv')

# Assuming 'Time Range' is in the format 'HH:MM-HH:MM'
# Split 'Time Range' into 'start_time' and 'end_time'
data[['start_time', 'end_time']] = data['Time Range'].str.split('-', expand=True)

# Convert these new columns into datetime format
data['start_time'] = pd.to_datetime(data['start_time'], format='%H:%M').dt.time
data['end_time'] = pd.to_datetime(data['end_time'], format='%H:%M').dt.time

# Now you can work with these datetime objects
print(data[['start_time', 'end_time']].head())


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pywt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from joblib import dump

# Load your data
data = pd.read_csv('C:/Users/benz/SynologyDrive/PycharmProjects/ToxitracAI/datasets/bpm_norm_rates.csv')

def map_condition(condition):
    """A function that map different conditions to normal or active"""
    if condition in ['normal', 'resting']:
        return 'normal'
    else:
        return 'active'

data['Category'] = data['Condition'].apply(map_condition)
label_encoder = LabelEncoder()
data['Category'] = label_encoder.fit_transform(data['Category'])

data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day

# Process Time Range into datetime
data[['start_time', 'end_time']] = data['Time Range'].str.split('-', expand=True)
data['start_time'] = pd.to_datetime(data['start_time'], format='%H:%M')
data['end_time'] = pd.to_datetime(data['end_time'], format='%H:%M')
data['duration_hours'] = (data['end_time'] - data['start_time']).dt.seconds / 3600

# Convert BPM values to numerics and normalize
data['BPM min'] = pd.to_numeric(data['BPM min'], errors='coerce')
data['BPM max'] = pd.to_numeric(data['BPM max'], errors='coerce')
scaler = StandardScaler()
data[['BPM min', 'BPM max']] = scaler.fit_transform(data[['BPM min', 'BPM max']])

In [3]:
# First, ensure 'BPM min' and 'BPM max' do not contain NaNs for DWT transformation
mask = data['BPM min'].notna() & data['BPM max'].notna()
print(f'Display mask data:\n{mask.head()}')
data_filtered = data[mask]
print(f'Display filtered data:\n{data_filtered.head()}')


In [4]:
# Recalculate DWT features on filtered data
cA_min, _ = pywt.dwt(data_filtered['BPM min'].values, 'db1')
cA_max, _ = pywt.dwt(data_filtered['BPM max'].values, 'db1')

data_filtered = data_filtered.iloc[:len(cA_min)]  # Adjust the dataframe to match the DWT output length

# Ensure that the number of samples in your features and labels are identical
# Now prepare your feature set
X_filtered = data_filtered[['BPM min', 'BPM max']]
X_stacked = np.column_stack((X_filtered, cA_min, cA_max))
y_filtered = data_filtered['Category']

# Now, X_stacked and y_filtered can be used for splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_stacked, y_filtered, test_size=0.4, random_state=42)

# Train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

print(model.score(X_test, y_test))  # Score method measures the accuracy of the model predictions

In [5]:
from sklearn.metrics import log_loss

# Predict probabilities on the test set
prob_predictions = model.predict(X_test)

# Calculate the log loss
loss = log_loss(y_test, prob_predictions)
print(loss)